In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.cluster as cluster
import time
import underworld as uw
import math
from underworld import function as fn
import glucifer
import numpy as np
import os
import matplotlib.pyplot as plt
import unsupported.scaling as sca
from unsupported.scaling import nonDimensionalize as nd
from decimal import *
from numpy.linalg import eig,solve,lstsq
#import hdbscan
%matplotlib inline
sns.set_context('poster')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.25, 's' : 80, 'linewidths':0}

uw.matplotlib_inline()
# use low resolution if running in serial

u = sca.UnitRegistry

# Define scale criteria
tempMin = 273.*u.degK 
tempMax = (1400.+ 273.)*u.degK
bodyforce = 3300 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2
velocity = 1.*u.centimeter/u.year
velocity = 1e-9*u.meter/u.second

KL = 100e3*u.meter
Kt = KL/velocity
KT = tempMax 
KM = bodyforce * KL**2 * Kt**2
K  = 1.*u.mole
lengthScale = 100e3

sca.scaling["[length]"] = KL
sca.scaling["[time]"] = Kt
sca.scaling["[mass]"]= KM
sca.scaling["[temperature]"] = KT
sca.scaling["[substance]"] = K

gravity = nd(9.81 * u.meter / u.second**2)
R = nd(8.3144621 * u.joule / u.mole / u.degK)
step=0

outputPath = os.path.join(os.path.abspath("."),"landsliding_F4C2e6Vis24Point200B/")

if uw.rank()==0:
    if not os.path.exists(outputPath):
        os.makedirs(outputPath)
uw.barrier()

fo = open(outputPath+"dicMesh"+str(step).zfill(4),'r')
meshDic=eval(fo.read())

'''
mesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q1/dQ0"),
                                 elementRes  = meshDic['elements'], 
                                 minCoord    = meshDic['minCoord'], 
                                 maxCoord    = meshDic['maxCoord'],
                                 ) 
'''
xRes=meshDic['elements'][0]
yRes=meshDic['elements'][1]
dim  = 2  
mesh = uw.mesh.FeMesh_Cartesian( elementType = (("Q1/dQ0")), 
                                 elementRes  = ( xRes, yRes), 
                                 ) 
mesh.load(outputPath+"mesh"+str(step).zfill(4))

minX = min(mesh.data[:,0])
maxX = max(mesh.data[:,0])
minY = min(mesh.data[:,1])
maxY = max(mesh.data[:,1])

mesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q1/dQ0"),
                                 elementRes  = (xRes, yRes), 
                                 minCoord    = (minX, minY), 
                                 maxCoord    = (maxX, maxY),
                                 periodic    = [False, False]) 

In [3]:
vis_factor = nd(1*u.pascal*u.second)
vel_factor = nd(1*u.centimeter/u.year)
stress_factor = nd(1*u.pascal)
strainRate_factor = nd(1/u.second)
temp_factor = nd(1*u.degK)

In [4]:
step = 50
vis= uw.mesh.MeshVariable(mesh=mesh,nodeDofCount=1)
vis.load(outputPath+'meshViscosity'+str(step).zfill(4))
#devStressFn = 2.0 * vis*strainRate_2ndInvariant

velField = uw.mesh.MeshVariable(mesh=mesh,nodeDofCount=2)
velField.load(outputPath+'velocityField'+str(step).zfill(4))

strainRateFn = fn.tensor.symmetric( velField.fn_gradient )
strainRate_2ndInvariant = fn.tensor.second_invariant(strainRateFn)

curl = fn.tensor.antisymmetric(velField.fn_gradient )
rotation = curl[1].evaluate(mesh)
#stress = devStressFn.evaluate(mesh)

swarmCopy = uw.swarm.Swarm( mesh=mesh )
swarmCopy.load(outputPath+'swarm'+str(step).zfill(4))
plasticStrainCopy = swarmCopy.add_variable("double",1)
plasticStrainCopy.load(outputPath+"plasticStrain"+str(step).zfill(4))
strainRate = strainRate_2ndInvariant.evaluate(swarmCopy)
vis1 = vis.evaluate(mesh)
'''
meshPlasticStrain= uw.mesh.MeshVariable( mesh, 1 )
projectorPlasticStrain = uw.utils.MeshVariable_Projection( meshPlasticStrain,plasticStrainCopy, type=1)
projectorPlasticStrain.solve()
'''
#strainRate = strainRate_2ndInvariant.evaluate(mesh)

'\nmeshPlasticStrain= uw.mesh.MeshVariable( mesh, 1 )\nprojectorPlasticStrain = uw.utils.MeshVariable_Projection( meshPlasticStrain,plasticStrainCopy, type=1)\nprojectorPlasticStrain.solve()\n'


velField = uw.mesh.MeshVariable(mesh=mesh,nodeDofCount=2)
vis= uw.mesh.MeshVariable(mesh=mesh,nodeDofCount=1)

velField.load(outputPath+'velocityField'+str(step).zfill(4))
vis.load(outputPath+'meshViscosity'+str(step).zfill(4))

stressTensor  = uw.mesh.MeshVariable( mesh=mesh,  nodeDofCount=6 )
strainRateFn = fn.tensor.symmetric(velField.fn_gradient )
devStressFn = 2.0 * strainRateFn
stress  = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=3*3 )
stressEig  = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=3 )
stressAngle  = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=3 )
angleLeft = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=4 )
angleRight = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=2 )
ratioAngle = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=2 )
theta = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=1 )
cosX = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=1 )
cosY = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=1 )
cosZ = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=1 )

# In[3]:

stress.data[:,0]=devStressFn[0].evaluate(mesh)[:,0]
stress.data[:,4]=devStressFn[1].evaluate(mesh)[:,0]
stress.data[:,8]=0.
stress.data[:,1]=devStressFn[2].evaluate(mesh)[:,0]
stress.data[:,3]=devStressFn[2].evaluate(mesh)[:,0]
stress.data[:,5]=0.
stress.data[:,7]=0.
stress.data[:,2]=0.
stress.data[:,6]=0.

# In[4]:

for index, item in enumerate(stress.data): 
    stressEig.data[index,:]= eig(np.reshape(stress.data[index,:],[3,3]))[0]    

# In[5]:

stressEig.data[:].sort()

## 0-min principle stress(compression);2-max principle stress(extension)
sigma=stressEig.data[:,0]
angleLeft.data[:,0]=stress.data[:,3]
angleLeft.data[:,1]=stress.data[:,6]
angleLeft.data[:,2]=stress.data[:,4]-sigma
angleLeft.data[:,3]=stress.data[:,7]
angleRight.data[:,0]=sigma-stress.data[:,0]
angleRight.data[:,1]=-stress.data[:,1]
#stressEig.data


for index, item in enumerate(angleLeft.data): 
    ratioAngle.data[index,:]=lstsq(np.reshape(angleLeft.data[index],[2,2]),angleRight.data[index])[0]

for index, item in enumerate(ratioAngle.data):
    cosX.data[index]=1./math.pow(1.+ratioAngle.data[index,1]*ratioAngle.data[index,1]+                                   ratioAngle.data[index,0]*ratioAngle.data[index,0],0.5)
    cosY.data[index]=cosX.data[index]*ratioAngle.data[index,0]
    cosZ.data[index]=cosX.data[index]*ratioAngle.data[index,1]   

sigma1 = 1
stressMax_Vector = uw.mesh.MeshVariable( mesh, 2 )
stressMax_Vector.data[:,0]=sigma1*cosX.evaluate(mesh)[:,0]
stressMax_Vector.data[:,1]=sigma1*cosY.evaluate(mesh)[:,0]
theta.data[:,0] = np.degrees(np.arctan(stressMax_Vector.data[:,1]/stressMax_Vector.data[:,0]))



In [ ]:
X = np.zeros(((plasticStrainCopy.data.size),2))
X[:,1] = plasticStrainCopy.data[:,0]*5000
X[:,0] = np.log10(strainRate[:,0]/strainRate_factor)


In [ ]:
X = np.zeros(((vis.data.size),2))
X[:,1] = np.log10(vis.data[:,0]/vis_factor)/10
X[:,0] = np.log10(rotation[:,0]/strainRate_factor+1e-15)*10

In [ ]:
import sklearn.cluster as cluster
kmeans = cluster.KMeans(n_clusters=4)
y_kmeans = kmeans.fit_predict(X)

plt.subplots(figsize=(10,5))


plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=10, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=10, alpha=0.5);


In [ ]:
plt.subplots(figsize=(10,5))
xcoord = swarmCopy.particleCoordinates.data[:,0]
ycoord = swarmCopy.particleCoordinates.data[:,1]
plt.scatter(xcoord,ycoord,c=y_kmeans, s=10, cmap='viridis')

In [ ]:
xcoord = mesh.data[:,0]
ycoord = mesh.data[:,1]
plt.scatter(xcoord,ycoord,c=y_kmeans, s=10, cmap='viridis')


In [ ]:
import sklearn.cluster as cluster
dbscan = cluster.DBSCAN(eps=0.5)
dbscan.fit(X)
y_dbscan = dbscan.fit_predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_dbscan, s=10, cmap='viridis')


In [ ]:
def plot_clusters(data, algorithm, args, kwds):
    start_time = time.time()
    labels = algorithm(*args, **kwds).fit_predict(data)
    end_time = time.time()
    palette = sns.color_palette('deep', np.unique(labels).max() + 1)
    colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
    plt.scatter(data.T[0], data.T[1], c=colors, **plot_kwds)
    frame = plt.gca()
    plt.title('Clusters found by {}'.format(str(algorithm.__name__)), fontsize=24)
    #plt.text('Clustering took {:.2f} s'.format(end_time - start_time), fontsize=14)
    return labels

In [ ]:
labels=plot_clusters(X, cluster.MeanShift, (0.175,), {'cluster_all':False})


In [ ]:
step = 110   
swarmCopy1 = uw.swarm.Swarm( mesh=mesh )
swarmCopy1.load(outputPath+"swarm"+str(step).zfill(4))
plasticStrainCopy1 = swarmCopy1.add_variable("double",1)
plasticStrainCopy1.load(outputPath+"plasticStrain"+str(step).zfill(4))
materialVariable1 = swarmCopy1.add_variable("double",1)
materialVariable1.load(outputPath+"materialVariable"+str(step).zfill(4))
index1=np.where(materialVariable1.data==0)
Mat0 = uw.swarm.Swarm ( mesh=mesh )
Mat0.add_particles_with_coordinates(swarmCopy1.particleCoordinates.data[index1[0]])
plasticStrain1  = Mat0.add_variable( dataType="double",  count=1 )

In [ ]:
step = 120   
swarmCopy2 = uw.swarm.Swarm( mesh=mesh )
swarmCopy2.load(outputPath+"swarm"+str(step).zfill(4))
plasticStrainCopy2 = swarmCopy2.add_variable("double",1)
plasticStrainCopy2.load(outputPath+"plasticStrain"+str(step).zfill(4))
materialVariable2 = swarmCopy2.add_variable("double",1)
materialVariable2.load(outputPath+"materialVariable"+str(step).zfill(4))


index2=np.where(materialVariable1.data==0)
Mat02 = uw.swarm.Swarm ( mesh=mesh )
Mat02.add_particles_with_coordinates(swarmCopy2.particleCoordinates.data[index2[0]])
plasticStrain2  = Mat02.add_variable( dataType="double",  count=1 )






In [ ]:
for index, coord in enumerate(Mat0.particleCoordinates.data):
    plasticStrain1.data[index] = plasticStrainCopy1.data[index1[0][index]]
for index, coord in enumerate(Mat0.particleCoordinates.data):
    plasticStrain2.data[index] = plasticStrainCopy2.data[index1[0][index]]

In [ ]:
plasticStrain_diff = plasticStrain2 - plasticStrain1

In [ ]:
plasticStrain_diff.evaluate((0.1, 0.1))

In [ ]:
fig = glucifer.Figure( title="time="+str(time)+"Myr",figsize=(800,500),quality=3,
                                    margin = 100, rulers = True, rulerticks= 5 )
    

surf1 = glucifer.objects.Points( Mat0, plasticStrain_diff,pointSize=5,colours="blue white red" )
fig.append(surf1)
fig.show()